In [1]:
import pandas as pd
import numpy as np
import datetime

data = pd.read_excel('TYLE227.xlsx', sheet_name='ГРП')
dummy = pd.read_excel('TYLE227.xlsx', sheet_name='МЭР')
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт
0,1985-08-01,1,NaN,NaN,NaN,NaN,NaN,"Б8, Б9"
1,1985-09-01,1,NaN,NaN,NaN,NaN,NaN,"Б8, Б9"
2,1985-10-01,1,NaN,NaN,NaN,NaN,NaN,"Б8, Б9"
3,1985-11-01,1,NaN,NaN,NaN,NaN,NaN,"Б8, Б9"
4,1985-12-01,1,NaN,NaN,NaN,NaN,NaN,"Б8, Б9"
...,...,...,...,...,...,...,...,...
889135,2021-09-01,UAG31Р,NaN,NaN,NaN,NaN,NaN,NaN
889136,2021-10-01,UAG31Р,NaN,NaN,NaN,NaN,NaN,NaN
889137,2021-11-01,UAG31Р,NaN,NaN,NaN,NaN,NaN,NaN
889138,2021-12-01,UAG31Р,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
date_column_name = r'Дата ВНР после ГС \\ ГРП \\ЗБГС'
target = 'Скважина №'
data['Скважина №'] = data.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
dummy['Скважина №'] = dummy.apply(
            lambda x:
            str(x['Скважина №']), axis=1)

In [3]:
            data['для_грп'] = data.apply(
                lambda x:
                x['Скважина №'].partition('_')[0] if '_' in str(x['Скважина №'])
                else x['Скважина №'], axis=1)
            no_grp = data.groupby('для_грп').agg(
                {'ГС/ННС': lambda x:
                x.tolist()}
            )
            no_grp.reset_index(inplace=True)
            no_grp['ГРП?'] = no_grp.apply(
                lambda x:
                0 if 'ГРП' not in str(x['ГС/ННС'])
                else 1, axis=1)
            no_grp = no_grp.drop(no_grp[no_grp['ГРП?'] == 1].index)
            spisok = no_grp['для_грп'].tolist()
            step = data.groupby('для_грп').agg(
                {date_column_name: lambda x:
                x.tolist()[0]
                if len(x.tolist()) == 1 else x.tolist()[1]}
            )
            step.reset_index(inplace=True)
            step = step[~step['для_грп'].isin(spisok)].reset_index(drop=True)
            vvv = step['для_грп'].tolist()

            step['Скважина №'] = vvv
            step = step.drop(['для_грп'], axis=1)
            dummy = dummy.merge(step, on=target)
            dummy[target] = dummy.apply(
                lambda x:
                str(x[target])
                + ('_ГРП' if x['Дата'] > x[date_column_name] else ''),
                axis=1
            )
            dummy = dummy.drop(date_column_name, axis=1)
            dummy.head(40)

KeyError: "Column(s) ['Дата ВНР после ГС \\\\\\\\ ГРП \\\\\\\\ЗБГС'] do not exist"

In [ ]:
step